In [1]:
import pymongo as pm
from pprint import pprint
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

client = pm.MongoClient('bigdatadb.polito.it',
                        ssl=True,
                        authSource = 'carsharing',
                        tlsAllowInvalidCertificates=True)
db = client['carsharing']
db.authenticate('ictts', 'Ictts16!')

permanentParkings = db['PermanentParkings']


In [2]:
start = datetime.fromisoformat('2017-01-01')
end = datetime.fromisoformat('2017-02-28')
start_hour = '08' #8:00
end_hour = '12' #12:00
city = "Vancouver"


In [3]:
# select all parking in the time period
pipeline_parking_in_period = [{"$match": {"city": city}},
                        {"$match": {"$and": [{"init_date": {"$gte": start}}, {"final_date": {"$lte": end}}]}},
                        {"$project": {
                                    "_id": 1, 
                                    "longitude":{"$arrayElemAt": [ "$loc.coordinates", 0]},
                                    "latitude":{"$arrayElemAt": [ "$loc.coordinates", 1]},
                                    "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]}, 
                                    "start_hour": {"$dateToString": { "format": "%H", "date": "$init_date" }},
                                    "end_hour": {"$dateToString": { "format": "%H", "date": "$final_date" }},
                                    }
                        },
                        {"$match": {"$and": [   {"duration": {"$gte": 5}}, 
                                                {"duration": {"$lte": 3 * 60}},
                                            ],
                                    "$or":  [   
                                            {"$and":[{"start_hour":{"$gte":start_hour}},{"start_hour":{"$lte":end_hour}}]},
                                            {"$and":[{"start_hour":{"$lte":start_hour}},{"end_hour":{"$gte":end_hour}}]}
                                                ]}
                        },
                        {"$group": {"_id": {"longitude": "$longitude", "latitude": "$latitude"}, 
                                    "number":{"$sum":1}}
                                },
                        ]
parking_in_period = permanentParkings.aggregate(pipeline_parking_in_period)

In [4]:
# count = 0
# try:
#     while parking_in_period.alive:
#         print(parking_in_period.next())
#         count+=1
#         if count>10:
#             break
# except:
#     pass

In [5]:
count = 0
longitude=[]
latitude =[]
counting = []
frame_id =[]
for parking in parking_in_period:
    frame_id.append(count)
    longitude.append(parking["_id"]["longitude"])
    latitude.append(parking["_id"]["latitude"])
    counting.append(parking["number"])
    count+=1

df = pd.DataFrame({
    'id':frame_id,
    'longitude':longitude,
    'latitude':latitude,
    'density':counting
})
df.to_csv("./van8_12.csv",index=False)